In [1]:
# import 모듈
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import koreanize_matplotlib
import seaborn as sns
import plotly.express as px

import warnings
warnings.simplefilter("ignore") # 엑셀 파일 불러오때 심플 오류 무시하는 기능

In [2]:
# 로우데이터 파일 불러오기
# 애초에 데이터를 많이 가공한 상태로 가져오면 수고를 덜 수 있음
df_1 = pd.read_excel("5대+범죄+발생현황_20221021174201.xlsx")
# 범위 밖에 있는 데이터를 지우기 위한 컬럼 및 인덱스 작업
# dr_col = ["2013", "2013.1", "2013.2", "2013.3", 
#           "2013.4", "2013.5", "2013.6", "2013.7", 
#           "2013.8", "2013.9", "2013.10", "2013.11"] #더 편한 방법은 없을까?
# df_1 = df_1.drop(columns = dr_col)
df_1 = df_1.drop(columns = "자치구별(1)", index = 0)

# 로우데이터에서 강간과 강제추행의 데이터만 가져오기 위한 작업
# 2010.*에서 *의 숫자만 바꿔주면 데이터 변형 가능
# 하지만 2020.10 데이터는 다른 방식으로 접근해야함 (float이 적용이 되지 않음)

y_sum = []
for x in range(7):
    yn = x + 4
    yy = str(2010 + int(yn))
    yy_1 = str(2010.1 + int(yn))
    y_sum.append(yy)
    y_sum.append(yy_1)

# str()을 사용하면 숫자를 문자열로 변경 가능
# str(y_sum)

# y_sum에 처음 인덱스에 "자치구별(2)"를 넣어주고 형식을 맞춘 후에
# df_1에 해당 컬럼값만 가져오게 하는 방법
y_sum.insert(0, "자치구별(2)")
df_2 = df_1[y_sum]

# 컬럼명을 정리하고 필요없는 인덱스는 드랍하고 리 인덱싱 작업
df_2.columns = ["자치구별", "2014년_발생", "2014년_검거", 
                "2015년_발생", "2015년_검거", "2016년_발생", "2016년_검거", 
                "2017년_발생", "2017년_검거", "2018년_발생", "2018년_검거", 
                "2019년_발생", "2019년_검거", "2020년_발생", "2020년_검거"]
df_2 = df_2.drop(index = 1)
df_2 = df_2.drop(index = 2)
df_2 = df_2.reset_index(drop = True)

# melt를 이용하여 데이터를 시각화하기에 좋도록 정리
df_3 = df_2.melt(id_vars = "자치구별", value_vars = df_2.columns[1:], 
                 value_name = "수치", var_name = "발생 / 검거")

# 파생변수를 만듬 (연도와 발생 / 검거에 대한 파생변수)
df_3["연도"] = pd.to_numeric(df_3["발생 / 검거"].str[:4], errors = 'coerce')
df_3["발생 / 검거"] = df_3["발생 / 검거"].str[6:]

# 그룹바이 하여 한눈에 파악하기 쉽게 처리 가능
df_4 = df_3.groupby(["연도", "자치구별", "발생 / 검거"])[["수치"]].mean()

# 소계가 있는 인덱스를 모두 드랍하는 방법
dr_ind = df_3[df_3["자치구별"] == "소계"].index
df_5 = df_3.drop(dr_ind)
df_5

# plotly histogram 시각화
# 단점은 facet_col을 사용하면 자치구별이 모두 보이지 않음
# !!! 그룹바이 한 값은 row가 하나로 잡히기 때문에 시각화 데이터로 쓰기가 매우 어렵다.
# 시각화 데이터로 사용이 용이한것은 melt한 데이터들이다.
# px.histogram(df_5, x = "자치구별", y = "수치", color = "발생 / 검거", histfunc = "avg")

자치구별 발생 / 검거      수치    연도
1    종로구      발생  5021.0  2014
2     중구      발생  5231.0  2014
3    용산구      발생  3799.0  2014
4    성동구      발생  3582.0  2014
5    광진구      발생  6268.0  2014
..   ...     ...     ...   ...
359  관악구      검거  3858.0  2020
360  서초구      검거  3052.0  2020
361  강남구      검거  5245.0  2020
362  송파구      검거  3544.0  2020
363  강동구      검거  2662.0  2020

[350 rows x 4 columns]

In [3]:
# 엑셀 형식으로 파일 저장하기
df_5.to_excel("합계_데이터후처리파일.xlsx", index = False)

In [7]:
df_5[df_5["자치구별"] == "송파구"]["수치"].sum()

70782.0